In [1]:
import sys, os
import numpy as np
import faiss
import torch
from make_index import MarkdownParser
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from sentence_transformers import CrossEncoder
from tqdm import tqdm

In [2]:
# Автоперезагрузка файлов
%load_ext autoreload
%autoreload 2

Set folder path

In [3]:
DATABASE_PATH = os.path.abspath("../../var/database")
DOCS_PATH = os.path.abspath("../../docs")
DEVICE = "cpu"

# Read model name
with open(os.path.join(DATABASE_PATH, "model.txt"), "r", encoding="utf-8") as f:
    EMBEDDINGS_MODEL_NAME = f.read()

In [4]:
def get_embedding():
    
    """
    Returns embedding
    """
    
    embeddings_model = HuggingFaceEmbeddings(
        model_name=EMBEDDINGS_MODEL_NAME,
        model_kwargs={"device": DEVICE}
    )
    vector_store = FAISS.load_local(
        DATABASE_PATH,
        embeddings=embeddings_model,
        allow_dangerous_deserialization=True
    )
    return embeddings_model, vector_store


embeddings_model, vector_store = get_embedding()

In [5]:
llm = ChatOllama(
    base_url="http://database_ollama:11434",
    model="qwen2.5:1.5b",
    temperature = 0,
)

In [11]:
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
#reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
#reranker = CrossEncoder('cross-encoder/stsb-TinyBERT-L-2')

In [ ]:
def find_docs(questions, k=3):
    
    """
    Find documents by questions
    """
    
    embeddings = np.array(embeddings_model.embed_documents(questions)).astype(np.float32)
    mean_embedding = embeddings.mean(axis=0)
    faiss.normalize_L2(mean_embedding.reshape(1, -1))
    docs = vector_store.similarity_search_by_vector(mean_embedding.tolist(), k)
    return embeddings, docs


def find_docs2(question, k=3):
    
    """
    Find documents by questions
    """
    
    embeddings = np.array(embeddings_model.embed_query(question)).astype(np.float32)
    faiss.normalize_L2(embeddings.reshape(1, -1))
    docs = vector_store.similarity_search_by_vector(embeddings.tolist(), k)
    return embeddings, docs


def print_docs(docs):
    for i, doc in enumerate(docs):
        print(f"Result {i+1}")
        print(f"ID: {doc.metadata['id']}")
        print(f"Content:\n{doc.page_content}")
        #print(f"Distance: {distance}")
        if i + 1 < len(docs):
            print("-" * 50)


def get_llm_question(llm, question, history):
        
    prompt = [
        ("system",
            "Ты помощник, который делает вопросы самодостаточными, добавляя контекст из истории диалога"
        ),
        ("human",
            f"История диалога:\n\n" + "\n".join(history) + "\n\n" +
            f"Переформулируй вопрос, на основе истории диалога, " +
                f"чтобы он был понятен без контекста: {question}. Вопрос:"
        ),
    ]
    
    print(str(prompt) + "\n")
    
    answer = llm.invoke(prompt).content
    
    print("Новый вопрос: " + answer + "\n")
    
    return answer


def get_context1(question, history, k=3):
    questions = history + [question]
    
    # Ищем релевантные документы
    _, docs = find_docs(questions, k)
    return docs


def get_context2(question, history, k=3):
    
    new_question = question
    
    # Переформулировка вопроса
    if len(history) > 0:
        new_question = get_llm_question(llm, question, history)
    
    # Ищем релевантные документы
    _, docs = find_docs([new_question], k)
    return docs


def rerank_docs(query, docs, k=5):
    
    """
    Переранжирует документы с помощью cross-encoder.
    """
    
    pairs = [(query, doc.page_content) for doc in docs]
    scores = reranker.predict(pairs)
    
    # Сортируем по убыванию релевантности
    ranked_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    ranked_docs = [doc for doc, score in ranked_docs]
    return ranked_docs[0:k]


history = [
    "Что такое облачная BAYRELL Cloud OS?"
]
question = "Как его установить на сервер?"
docs = get_context2(question, history, k=5)
#docs = rerank_docs(question, docs, k=3)
print_docs(docs)

In [14]:
def markdown_test():
    file_name = os.path.join(DOCS_PATH, "ru/cloud-os/install-desktop.md")
    with open(file_name, "r", encoding="utf-8") as f:
        parser = MarkdownParser()
        parser.parse(f.read())
        for i, block in enumerate(parser.blocks):
            print(f"Result {i+1}")
            print(block["content"])
            if i + 1 < len(parser.blocks):
                print("-" * 50)

In [ ]:
# Запрос и кандидаты
query = "Что такое Облачная ОС?"
documents = [
    "Облачная ОС — это операционная система, работающая в облаке.",
    "Компьютерная сеть — это соединение устройств для обмена данными.",
    "Linux — это операционная система с открытым кодом."
]

# Формируем пары (query, document)
pairs = [(query, doc) for doc in documents]

# Предсказываем релевантность (чем выше — тем лучше)
scores = reranker.predict(pairs)

# Сортируем документы по убыванию оценки
ranked_docs = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

# Выводим результаты
for i, (doc, score) in enumerate(ranked_docs):
    print(f"Rank {i+1}: {doc} (Score: {score:.4f})")
